In [ ]:
import torch
print(torch.__version__)
print(torch.version.cuda)

2.2.1+cu121
12.1


In [ ]:
!pip install torch-scatter==2.1.2 -f https://pytorch-geometric.com/whl/torch-2.2.1+cu121.html

Looking in links: https://pytorch-geometric.com/whl/torch-2.2.1+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 65.0 MB/s eta 0:00:00


In [ ]:
!pip install torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 3.5 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Model

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.nn import Sequential as Seq, Linear as Lin, ReLU
from torch.nn import TransformerEncoderLayer, TransformerEncoder
from torch.nn.parameter import Parameter
from torch_scatter import scatter_mean
from torch_geometric.nn import MetaLayer

TIME_WINDOW = 24
PRED_LEN = 6


In [ ]:
class Model(nn.Module):
	def __init__(self,mode,encoder,w_init,w,x_em,date_em,loc_em,edge_h,gnn_h,gnn_layer,city_num,group_num,pred_step,device):
		super(Model, self).__init__()
		self.device = device
		self.mode = mode
		self.encoder = encoder
		self.w_init = w_init
		self.city_num = city_num
		self.group_num = group_num
		self.edge_h = edge_h
		self.gnn_layer = gnn_layer
		self.pred_step = pred_step

		if self.encoder == 'self':
			self.encoder_layer = TransformerEncoderLayer(8, nhead=4, dim_feedforward=256) ### to extract the features ( self attention , 특정시점의 도시의 8개의 특성끼리 attention )
			# self.x_embed = Lin(8, x_em)
			self.x_embed = Lin(TIME_WINDOW*8, x_em) ### to obtain city representation

		elif self.encoder == 'lstm':
			self.input_LSTM = nn.LSTM(8,x_em,num_layers=1,batch_first=True)
		if self.w_init == 'rand':
			self.w = Parameter(torch.randn(city_num,group_num).to(device,non_blocking=True),requires_grad=True)
		elif self.w_init == 'group':
			self.w = Parameter(w,requires_grad=True)
		self.loc_embed = Lin(2, loc_em)
		self.u_embed1 = nn.Embedding(12, date_em) #month
		self.u_embed2 = nn.Embedding(7, date_em) #week
		self.u_embed3 = nn.Embedding(24, date_em) #hour
		self.edge_inf = Seq(Lin(x_em*2+date_em*3+loc_em*2,edge_h),ReLU(inplace=True))
		self.group_gnn = nn.ModuleList([NodeModel(x_em+loc_em,edge_h,gnn_h)])
		for i in range(self.gnn_layer-1):
			self.group_gnn.append(NodeModel(gnn_h,edge_h,gnn_h))
		self.global_gnn = nn.ModuleList([NodeModel(x_em+gnn_h,1,gnn_h)])
		for i in range(self.gnn_layer-1):
			self.global_gnn.append(NodeModel(gnn_h,1,gnn_h))
		if self.mode == 'ag':
			self.decoder = DecoderModule(x_em,edge_h,gnn_h,gnn_layer,city_num,group_num,device)
			self.predMLP = Seq(Lin(gnn_h,16),ReLU(inplace=True),Lin(16,1),ReLU(inplace=True))
		if self.mode == 'full':
			self.decoder = DecoderModule(x_em,edge_h,gnn_h,gnn_layer,city_num,group_num,device)
			self.predMLP = Seq(Lin(gnn_h,16),ReLU(inplace=True),Lin(16,self.pred_step),ReLU(inplace=True))
		self.TemporalAggregateMLP = Seq(Lin(gnn_h+8,gnn_h),ReLU(inplace=True))

	def batchInput(self,x,edge_w,edge_index):
		sta_num = x.shape[1]
		x = x.reshape(-1,x.shape[-1])
		edge_w = edge_w.reshape(-1,edge_w.shape[-1])
		for i in range(edge_index.size(0)):
			edge_index[i,:] = torch.add(edge_index[i,:], i*sta_num)
		# print(edge_index.shape)
		edge_index = edge_index.transpose(0,1)
		# print(edge_index.shape)
		edge_index = edge_index.reshape(2,-1)
		return x, edge_w, edge_index

	def forward(self,x,u,edge_index,edge_w,loc):
		x = x.reshape(-1,x.shape[2],x.shape[3])
		if self.encoder == 'self':
			# [S,B,E]
			# print(x.shape)
			x = x.transpose(0,1)
			x = self.encoder_layer(x)
			x = x.transpose(0,1)
			x2 = x.reshape(-1,self.city_num,x.shape[1],x.shape[2])  ## (batch size, num of city, 24h, 8features )
			# print(x.shape)
			x = x.reshape(-1,self.city_num,TIME_WINDOW*x.shape[-1])
			x = self.x_embed(x)
			# x = x.reshape(-1,self.city_num,TIME_WINDOW,x.shape[-1])
			# x = torch.max(x,dim=-2).values
			# print(x.shape)
		elif self.encoder == 'lstm':
			_,(x,_) = self.input_LSTM(x)
			x = x.reshape(-1,self.city_num,x.shape[-1])
			# print(x.shape)

		x2 = torch.index_select(x2, dim=2, index=torch.tensor([3, 7, 11, 15, 19, 23]).to('cuda')) ## ( batch size, num of city, selected 6h, 8features )
		x2 = x2.transpose(1,2) ## ( batch size, selected 6h , num of city, 8features ) [batch size, 6, 209, 8]

		h_other5 = x2[:, :-1, :, :]  # [batch size, 5, 209, 8]
		h_other5 = h_other5.reshape(-1,h_other5.shape[1]*h_other5.shape[2],h_other5.shape[3])  # [batch size, 1045, 8]

		h24 = x2[:, -1:, :, :]  # [batch size, 1, 209, 8]
		h24 = h24.reshape(-1,h24.shape[2],h24.shape[3]) # [batch size, 209, 8]
		h24 = h24.transpose(1,2)

		attention_scores = torch.matmul(h_other5, h24)   # [batch size, 1045, 209]
		attention_scores = attention_scores.reshape(-1,5,209,attention_scores.shape[2])  # [batch size, 5, 209, 209]
		attention_weights = F.softmax(attention_scores, dim=1)  # [batch size, 5, 209, 209]

		h_other5 = h_other5.reshape(-1,5,209,h_other5.shape[2]) # [batch size, 5, 209, 8]
		h_other5 = h_other5.transpose(2,3) # [batch size, 5, 8, 209]

		attention_weighted_sum = torch.matmul(h_other5, attention_weights) # [batch size, 5, 8, 209]
		attention_weighted_sum = attention_weighted_sum.transpose(2,3)
		x2 = torch.sum(attention_weighted_sum, dim=1)  # [batch size, 209, 8]


		# graph pooling
		# print(self.w[10])
		w = F.softmax(self.w)
		w1 = w.transpose(0,1)
		w1 = w1.unsqueeze(dim=0)
		w1 = w1.repeat_interleave(x.size(0), dim=0)
		# print(w.shape,x.shape)
		# print(loc.shape)
		loc = self.loc_embed(loc)
		x_loc = torch.cat([x,loc],dim=-1)
		g_x = torch.bmm(w1,x_loc)
		# print(g_x.shape)

		# group gnn
		u_em1 = self.u_embed1(u[:,0])
		u_em2 = self.u_embed2(u[:,1])
		u_em3 = self.u_embed3(u[:,2])
		u_em = torch.cat([u_em1,u_em2,u_em3],dim=-1)
		# print(u_em.shape)
		for i in range(self.group_num):
			for j in range(self.group_num):
				if i == j: continue
				g_edge_input = torch.cat([g_x[:,i],g_x[:,j],u_em],dim=-1)
				tmp_g_edge_w = self.edge_inf(g_edge_input)
				tmp_g_edge_w = tmp_g_edge_w.unsqueeze(dim=0)
				tmp_g_edge_index = torch.tensor([i,j]).unsqueeze(dim=0).to(self.device,non_blocking=True)
				if i == 0 and j == 1:
					g_edge_w = tmp_g_edge_w
					g_edge_index = tmp_g_edge_index
				else:
					g_edge_w = torch.cat([g_edge_w,tmp_g_edge_w],dim=0)
					g_edge_index = torch.cat([g_edge_index,tmp_g_edge_index],dim=0)
		# print(g_edge_w.shape,g_edge_index.shape)
		g_edge_w = g_edge_w.transpose(0,1)
		g_edge_index = g_edge_index.unsqueeze(dim=0)
		g_edge_index = g_edge_index.repeat_interleave(u_em.shape[0],dim=0)
		g_edge_index = g_edge_index.transpose(1,2)
		# print(g_x.shape,g_edge_w.shape,g_edge_index.shape)
		g_x, g_edge_w, g_edge_index = self.batchInput(g_x, g_edge_w, g_edge_index)
		# print(g_x.shape,g_edge_w.shape,g_edge_index.shape)
		for i in range(self.gnn_layer):
			g_x = self.group_gnn[i](g_x,g_edge_index,g_edge_w)

		g_x = g_x.reshape(-1,self.group_num,g_x.shape[-1])
		# print(g_x.shape,self.w.shape)
		w2 = w.unsqueeze(dim=0)
		w2 = w2.repeat_interleave(g_x.size(0), dim=0)
		new_x = torch.bmm(w2,g_x)
		# print(new_x.shape,x.shape)
		new_x = torch.cat([x,new_x],dim=-1)
		edge_w = edge_w.unsqueeze(dim=-1)
		# print(new_x.shape,edge_w.shape,edge_index.shape)
		new_x, edge_w, edge_index = self.batchInput(new_x, edge_w, edge_index)
		# print(new_x.shape,edge_w.shape,edge_index.shape)
		for i in range(self.gnn_layer):
			new_x = self.global_gnn[i](new_x,edge_index,edge_w)
		# print(new_x.shape)

		x2 = x2.reshape(-1,x2.shape[-1])
		new_x = torch.cat([x2,new_x],dim=-1)
		new_x = self.TemporalAggregateMLP(new_x)

		if self.mode == 'ag':
			for i in range(self.pred_step):
				new_x = self.decoder(new_x,self.w,g_edge_index,g_edge_w,edge_index,edge_w)
				tmp_res = self.predMLP(new_x)
				tmp_res = tmp_res.reshape(-1,self.city_num)
				tmp_res = tmp_res.unsqueeze(dim=-1)
				if i == 0:
					res = tmp_res
				else:
					res = torch.cat([res,tmp_res],dim=-1)
		if self.mode == 'full':
			new_x = self.decoder(new_x,self.w,g_edge_index,g_edge_w,edge_index,edge_w)
			res = self.predMLP(new_x)
			res = res.reshape(-1,self.city_num,self.pred_step)

		# print(res.shape)
		return res

class DecoderModule(nn.Module):
	def __init__(self,x_em,edge_h,gnn_h,gnn_layer,city_num,group_num,device):
		super(DecoderModule, self).__init__()
		self.device = device
		self.city_num = city_num
		self.group_num = group_num
		self.gnn_layer = gnn_layer
		self.x_embed = Lin(gnn_h, x_em)
		self.group_gnn = nn.ModuleList([NodeModel(x_em,edge_h,gnn_h)])
		for i in range(self.gnn_layer-1):
			self.group_gnn.append(NodeModel(gnn_h,edge_h,gnn_h))
		self.global_gnn = nn.ModuleList([NodeModel(x_em+gnn_h,1,gnn_h)])
		for i in range(self.gnn_layer-1):
			self.global_gnn.append(NodeModel(gnn_h,1,gnn_h))

	def forward(self,x,trans_w,g_edge_index,g_edge_w,edge_index,edge_w):
		x = self.x_embed(x)
		x = x.reshape(-1,self.city_num,x.shape[-1])
		w = Parameter(trans_w,requires_grad=False).to(self.device,non_blocking=True)
		w1 = w.transpose(0,1)
		w1 = w1.unsqueeze(dim=0)
		w1 = w1.repeat_interleave(x.size(0), dim=0)
		g_x = torch.bmm(w1,x)
		g_x = g_x.reshape(-1,g_x.shape[-1])
		for i in range(self.gnn_layer):
			g_x = self.group_gnn[i](g_x,g_edge_index,g_edge_w)
		g_x = g_x.reshape(-1,self.group_num,g_x.shape[-1])
		w2 = w.unsqueeze(dim=0)
		w2 = w2.repeat_interleave(g_x.size(0), dim=0)
		new_x = torch.bmm(w2,g_x)
		new_x = torch.cat([x,new_x],dim=-1)
		new_x = new_x.reshape(-1,new_x.shape[-1])
		# print(new_x.shape,edge_w.shape,edge_index.shape)
		for i in range(self.gnn_layer):
			new_x = self.global_gnn[i](new_x,edge_index,edge_w)

		return new_x


class NodeModel(torch.nn.Module):
    def __init__(self,node_h,edge_h,gnn_h):
        super(NodeModel, self).__init__()
        self.node_mlp_1 = Seq(Lin(node_h+edge_h,gnn_h), ReLU(inplace=True))
        self.node_mlp_2 = Seq(Lin(node_h+gnn_h,gnn_h), ReLU(inplace=True))

    def forward(self, x, edge_index, edge_attr):
        # x: [N, F_x], where N is the number of nodes.
        # edge_index: [2, E] with max entry N - 1.
        # edge_attr: [E, F_e]
        row, col = edge_index
        out = torch.cat([x[row], edge_attr], dim=1)
        out = self.node_mlp_1(out)
        out = scatter_mean(out, col, dim=0, dim_size=x.size(0))
        out = torch.cat([x, out], dim=1)
        return self.node_mlp_2(out)

### Dataset

In [ ]:
import numpy as np
import pandas as pd
import os

import torch
import torch.utils.data as Data


path = '/content/drive/MyDrive/SKKU_AI_proj/project/GAGNN/GAGNN/data' ## set dataset path

In [ ]:
class trainDataset(Data.Dataset):
	def __init__(self, transform=None, train=True):
		self.x = np.load(os.path.join(path,'train_x.npy'),allow_pickle=True)
		self.u = np.load(os.path.join(path,'train_u.npy'),allow_pickle=True)
		self.y = np.load(os.path.join(path,'train_y.npy'),allow_pickle=True)
		self.edge_w = np.load(os.path.join(path,'edge_w.npy'),allow_pickle=True)
		self.edge_index = np.load(os.path.join(path,'edge_index.npy'),allow_pickle=True)
		self.loc = np.load(os.path.join(path,'loc_filled.npy'),allow_pickle=True)
		self.loc = self.loc.astype(np.float64)


	def __getitem__(self, index):
		x = torch.FloatTensor(self.x[index])
		x = x.transpose(0,1)
		y = torch.FloatTensor(self.y[index])
		y = y.transpose(0,1)
		u = torch.tensor(self.u[index])
		edge_index = torch.tensor(self.edge_index)
		# edge_index = edge_index.expand((x.size[0],edge_index.size[0],edge_index.size[1]))
		edge_w = torch.FloatTensor(self.edge_w)
		# edge_w = edge_w.expand((x.size[0],edge_w.size[0]))
		loc = torch.FloatTensor(self.loc)

		return [x,u,y,edge_index,edge_w,loc]

	def __len__(self):
		return self.x.shape[0]

class valDataset(Data.Dataset):
	def __init__(self, transform=None, train=True):
		self.x = np.load(os.path.join(path,'val_x.npy'),allow_pickle=True)
		self.u = np.load(os.path.join(path,'val_u.npy'),allow_pickle=True)
		self.y = np.load(os.path.join(path,'val_y.npy'),allow_pickle=True)
		self.edge_w = np.load(os.path.join(path,'edge_w.npy'),allow_pickle=True)
		self.edge_index = np.load(os.path.join(path,'edge_index.npy'),allow_pickle=True)
		self.loc = np.load(os.path.join(path,'loc_filled.npy'),allow_pickle=True)
		self.loc = self.loc.astype(np.float64)


	def __getitem__(self, index):
		x = torch.FloatTensor(self.x[index])
		x = x.transpose(0,1)
		y = torch.FloatTensor(self.y[index])
		y = y.transpose(0,1)
		u = torch.tensor(self.u[index])
		edge_index = torch.tensor(self.edge_index)
		# edge_index = edge_index.expand((x.size[0],edge_index.size[0],edge_index.size[1]))
		edge_w = torch.FloatTensor(self.edge_w)
		# edge_w = edge_w.expand((x.size[0],edge_w.size[0]))
		loc = torch.FloatTensor(self.loc)

		return [x,u,y,edge_index,edge_w,loc]

	def __len__(self):
		return self.x.shape[0]

class testDataset(Data.Dataset):
	def __init__(self, transform=None, train=True):
		self.x = np.load(os.path.join(path,'test_x.npy'),allow_pickle=True)
		self.u = np.load(os.path.join(path,'test_u.npy'),allow_pickle=True)
		self.y = np.load(os.path.join(path,'test_y.npy'),allow_pickle=True)
		self.edge_w = np.load(os.path.join(path,'edge_w.npy'),allow_pickle=True)
		self.edge_index = np.load(os.path.join(path,'edge_index.npy'),allow_pickle=True)
		self.loc = np.load(os.path.join(path,'loc_filled.npy'),allow_pickle=True)
		self.loc = self.loc.astype(np.float64)


	def __getitem__(self, index):
		x = torch.FloatTensor(self.x[index])
		x = x.transpose(0,1)
		y = torch.FloatTensor(self.y[index])
		y = y.transpose(0,1)
		u = torch.tensor(self.u[index])
		edge_index = torch.tensor(self.edge_index)
		# edge_index = edge_index.expand((x.size[0],edge_index.size[0],edge_index.size[1]))
		edge_w = torch.FloatTensor(self.edge_w)
		# edge_w = edge_w.expand((x.size[0],edge_w.size[0]))
		loc = torch.FloatTensor(self.loc)

		return [x,u,y,edge_index,edge_w,loc]

	def __len__(self):
		return self.x.shape[0]

### Train

In [ ]:
import time
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as Data
from sklearn.cluster import KMeans
from torch_geometric.nn import MetaLayer

import argparse

In [ ]:
class Args:
    device = 'cuda'
    mode = 'full'
    encoder = 'self'
    w_init = 'rand'
    mark = ''
    run_times = 1
    epoch = 1
    batch_size = 64
    w_rate = 50
    city_num = 209
    group_num = 15
    gnn_h = 32
    gnn_layer = 2
    x_em = 32
    date_em = 4
    loc_em = 12
    edge_h = 12
    lr = 0.001
    wd = 0.001
    pred_step = 6

args = Args()
print(args.device, args.mode, args.encoder, args.w_init, args.mark, args.run_times, args.epoch, args.batch_size, args.w_rate, args.city_num, args.group_num, args.gnn_h, args.gnn_layer, args.x_em, args.date_em, args.loc_em, args.edge_h, args.lr, args.wd, args.pred_step)

cuda full self rand  1 1 64 50 209 15 32 2 32 4 12 12 0.001 0.001 6


In [ ]:
train_dataset = trainDataset()
val_dataset = valDataset()
test_dataset = testDataset()
print(len(train_dataset)+len(val_dataset)+len(test_dataset))
train_loader = Data.DataLoader(train_dataset, batch_size=args.batch_size,
    shuffle=True, num_workers=8, pin_memory=True)
val_loader = Data.DataLoader(val_dataset, batch_size=args.batch_size,
    shuffle=False, num_workers=8, pin_memory=True)
test_loader = Data.DataLoader(test_dataset, batch_size=args.batch_size,
    shuffle=False, num_workers=8, pin_memory=True)

device = args.device
# city_index = [0,2,30,32,43]

20370


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
for _ in range(args.run_times):
	start = time.time()

	w = None
	if args.w_init == 'group':
		city_loc = np.load(os.path.join(path,'loc_filled.npy'),allow_pickle=True)
		kmeans = KMeans(n_clusters=args.group_num, random_state=0).fit(city_loc)
		group_list = kmeans.labels_.tolist()
		w = np.random.randn(args.city_num,args.group_num)
		w = w * 0.1
		for i in range(len(group_list)):
			w[i,group_list[i]] = 1.0
		w = torch.FloatTensor(w).to(device,non_blocking=True)

	city_model = Model(args.mode,args.encoder,args.w_init,w,args.x_em,args.date_em,args.loc_em,args.edge_h,args.gnn_h,
			args.gnn_layer,args.city_num,args.group_num,args.pred_step,device).to(device)
	city_num = sum(p.numel() for p in city_model.parameters() if p.requires_grad)
	print('city_model:', 'Trainable,', city_num)
	# print(city_model)
	criterion = nn.L1Loss(reduction = 'sum')
	all_params = city_model.parameters()
	w_params = []
	other_params = []
	for pname, p in city_model.named_parameters():
		if pname == 'w':
			w_params += [p]
	params_id = list(map(id, w_params))
	other_params = list(filter(lambda p: id(p) not in params_id, all_params))
	# print(len(w_params),len(other_params))
	optimizer = torch.optim.Adam([
        {'params': other_params},
        {'params': w_params, 'lr': args.lr * args.w_rate}
    ], lr=args.lr, weight_decay=args.wd)

	val_loss_min = np.inf
	for epoch in range(args.epoch):
		for i,data in enumerate(train_loader):
			data = [item.to(device,non_blocking=True) for item in data]
			x,u,y,edge_index,edge_w,loc = data
			outputs = city_model(x,u,edge_index,edge_w,loc)
			loss = criterion(y,outputs)
			city_model.zero_grad()
			loss.backward()
			optimizer.step()

			if epoch % 10 == 0 and i % 100 == 0:
				print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
						.format(epoch, args.epoch, i, int(len(train_dataset)/args.batch_size), loss.item()))

		if epoch % 5 == 0:
			with torch.no_grad():
				val_loss = 0
				for j, data_val in enumerate(val_loader):
					data_val = [item.to(device,non_blocking=True) for item in data_val]
					x_val,u_val,y_val,edge_index_val,edge_w_val,loc_val = data_val
					outputs_val = city_model(x_val,u_val,edge_index_val,edge_w_val,loc_val)
					batch_loss = criterion(y_val,outputs_val)
					val_loss += batch_loss.item()
				print('Epoch:',epoch,', val_loss:',val_loss)
				if val_loss < val_loss_min:
					torch.save(city_model.state_dict(),args.encoder+'_para_'+args.mark+'.ckpt')
					val_loss_min = val_loss
					print('parameters have been updated during epoch ',epoch)

	mae_loss = torch.zeros(args.city_num,args.pred_step).to(device)
	rmse_loss = torch.zeros(args.city_num,args.pred_step).to(device)

	def cal_loss(outputs,y):
		global mae_loss, rmse_loss
		temp_loss = torch.abs(outputs-y)
		mae_loss = torch.add(mae_loss,temp_loss.sum(dim=0))

		temp_loss = torch.pow(temp_loss,2)
		rmse_loss = torch.add(rmse_loss,temp_loss.sum(dim=0))


	with torch.no_grad():
		city_model.load_state_dict(torch.load(args.encoder+'_para_'+args.mark+'.ckpt'))
		w_weight = city_model.state_dict()['w']
		w_weight = F.softmax(w_weight)
		_,w_weight = torch.max(w_weight,dim=-1)
		print(w_weight.cpu().tolist())

		for i, data in enumerate(test_loader):
			data = [item.to(device,non_blocking=True) for item in data]
			x,u,y,edge_index,edge_w,loc = data
			outputs = city_model(x,u,edge_index,edge_w,loc)
			cal_loss(outputs,y)

		mae_loss = mae_loss/(len(test_dataset))
		rmse_loss = rmse_loss/(len(test_dataset))
		mae_loss = mae_loss.mean(dim=0)
		rmse_loss = rmse_loss.mean(dim=0)

		end = time.time()
		print('Running time: %s Seconds'%(end-start))

		mae_loss = mae_loss.cpu()
		rmse_loss = rmse_loss.cpu()

		print('mae:', np.array(mae_loss))
		print('rmse:', np.sqrt(np.array(rmse_loss)))

		# for i, data in enumerate(Data.DataLoader(test_dataset, batch_size=1,shuffle=False, pin_memory=True)):
		# 	data = [item.to(device,non_blocking=True) for item in data]
		# 	x,u,y,edge_index,edge_w,loc = data
		# 	outputs = city_model(x,u,edge_index,edge_w,loc)
		# 	if i == 305:
		# 		print(x[:,0])
		# 		print(outputs[:,0])

city_model: Trainable, 50025


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
<ipython-input-7-e8ca64cf9163>:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  w = F.softmax(self.w)


Epoch [0/1], Step [0/222], Loss: 5474274.0000
Epoch [0/1], Step [100/222], Loss: 2601227.0000
Epoch [0/1], Step [200/222], Loss: 2475348.2500


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Epoch: 0 , val_loss: 55513414.75
parameters have been updated during epoch  0
[3, 13, 1, 13, 14, 7, 13, 13, 13, 2, 8, 3, 5, 7, 13, 13, 8, 13, 13, 11, 3, 2, 11, 7, 13, 11, 3, 9, 5, 5, 4, 13, 12, 11, 11, 13, 3, 1, 9, 11, 13, 4, 8, 13, 3, 13, 13, 8, 3, 13, 7, 13, 11, 3, 13, 9, 11, 11, 9, 7, 3, 3, 13, 11, 1, 11, 12, 5, 11, 1, 3, 11, 3, 11, 9, 8, 11, 3, 8, 3, 13, 8, 13, 11, 8, 13, 11, 3, 7, 11, 1, 2, 11, 13, 8, 8, 13, 11, 3, 11, 1, 1, 5, 12, 8, 14, 13, 8, 7, 7, 13, 9, 13, 8, 13, 9, 3, 3, 9, 3, 8, 6, 8, 10, 13, 12, 3, 13, 12, 3, 8, 14, 13, 9, 9, 8, 8, 13, 7, 8, 14, 8, 3, 13, 11, 3, 13, 7, 1, 11, 2, 7, 14, 8, 3, 11, 13, 11, 11, 11, 3, 1, 7, 1, 3, 3, 13, 13, 3, 13, 11, 7, 14, 12, 9, 8, 8, 5, 8, 11, 5, 11, 3, 3, 11, 6, 9, 7, 4, 12, 12, 2, 8, 8, 13, 8, 8, 13, 4, 7, 11, 11, 11, 11, 13, 8, 13, 6, 3]


<ipython-input-13-0f1babfe465e>:80: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  w_weight = F.softmax(w_weight)


Running time: 69.3779878616333 Seconds
mae: [73.84241  29.602959 29.521553 29.625525 29.709352 29.756117]
rmse: [91.26294  45.754723 45.77058  45.881382 45.912903 46.019096]
